In [7]:
%pip install tqdm ipython


Note: you may need to restart the kernel to use updated packages.


In [10]:
import tensorflow as tf 
from tensorflow import keras 
import numpy as np 
from tqdm import tqdm 
from Ipython import display 
print('Tensorflow version : ',tf.__vertsion__)

ModuleNotFoundError: No module named 'Ipython'